In [ ]:
import pandas as pd
import numpy as np
import cirq
import matplotlib.pyplot as plt
import requests
import json
import fredpy as fp
#from fredapi import Fred

fp.api_key = fp.load_api_key('cucumber.txt')
#fp = Fred(api_key=fp.api_key)

api_key = fp.api_key

# FRED series ID to retrieve data for
series_id = 'SP500'

# Number of steps in the prediction
num_steps = 30

# Request data from FRED API
url = f'https://api.stlouisfed.org/fred/series/observations?series_id={series_id}&api_key={api_key}&file_type=json'
response = requests.get(url)
data = json.loads(response.text)['observations']
df = pd.DataFrame(data)

# Clean data and prepare for prediction
df['date'] = pd.to_datetime(df['date'])
df['value'] = pd.to_numeric(df['value'], errors='coerce')
df.dropna(inplace=True)
df.set_index('date', inplace=True)
start_date = df.index[-1]
end_date = start_date + pd.DateOffset(days=num_steps)
dates = pd.date_range(start_date, end_date)

# Prepare quantum circuit and simulator
# Update qubit initialization to match circuit definition
qubit = cirq.GridQubit(0, 0)

# Update circuit definition to measure qubit in the X basis
circuit = cirq.Circuit(cirq.X(qubit), cirq.measure(qubit, key='0'))


def generate_predictions(steps):
    predictions = []
    for i in range(steps):
        result = simulator.run(circuit, repetitions=100)
        counts = result.histogram(key='0,')[0]
        if 1 in counts:
            predictions.append(counts[1] / sum(counts))
        else:
            predictions.append(0)
    if not predictions:
        raise ValueError("No predictions generated.")
    # Fit predictions to a log-linear power law function
    x = np.arange(1, steps+1)
    y = np.log(predictions)
    popt, pcov = curve_fit(lambda t,a,b: a*np.log(t)+b,  x,  y)
    a, b = popt
    predictions_fit = np.exp(a*np.log(x) + b)
    return predictions_fit


# Define function to generate predictions
num_steps = 20
predictions = generate_predictions(num_steps)

plt.plot(predictions, label='Predictions')
plt.plot(predictions_fit, label='Fitted curve')
plt.legend()
plt.show()


In [ ]:
import cirq
import matplotlib.pyplot as plt
import pandas as pd
import scipy

def plot_stock_price_projection():
    # Read CSV file and extract date and value columns
    df = pd.read_csv("input.csv", usecols=[0,1], parse_dates=[0])
    df = df.dropna()  # Remove rows with empty values
    dates = df['date']
    values = df['value']

    # Define log linear power law function for projection
    def log_linear_power_law(x, a, b, c):
        return a * x**b + c

    # Fit function to historical data
    popt, pcov = scipy.optimize.curve_fit(log_linear_power_law, range(len(values)), values)

    # Generate projection values for 2 years
    projection_range = range(len(values), len(values) + 365*2)
    projection_values = [log_linear_power_law(x, *popt) for x in projection_range]

    # Plot historical data and projection
    fig, ax = plt.subplots()
    ax.plot(dates, values, label='Historical')
    ax.plot(pd.date_range(start=dates.iloc[-1], periods=len(projection_values), freq='D'), projection_values, label='Projection')
    ax.set_xlabel('Date')
    ax.set_ylabel('Price')
    ax.set_title('Stock Price Projection')
    ax.legend()
    plt.show()
plot_stock_price_projection()


In [ ]:
from cirq.contrib.svg import SVGCircuit
import pandas as pd
import numpy as np
import cirq
import matplotlib.pyplot as plt
import requests
import json
import fredpy as fp



fp.api_key = fp.load_api_key('cucumber.txt')
fp = Fred(api_key=fp.api_key)

api_key = fp.api_key

# FRED series ID to retrieve data for
series_id = 'SP500'

# Define the start and end date of the chart
start_date = '2020-01-01'
#end_date = '2023-03-31' This script does not need this line but could be worked into code


# Set up the circuit
qubits = [cirq.GridQubit(0, i) for i in range(3)]
circuit = cirq.Circuit()
circuit.append(cirq.H(q) for q in qubits)
circuit.append(cirq.measure(*qubits, key='m'))

# Set up the simulator
simulator = cirq.Simulator()

# Simulate the circuit
num_shots = 1000
result = simulator.run(circuit, repetitions=num_shots)

# Get the probabilities of the final state
num_qubits = len(qubits)
probs = []
for i in range(num_qubits):
    if i in result.multi_measurement_histogram_dict():
        probs.append(result.multi_measurement_histogram_dict()[i][0]/num_shots)
    else:
        probs.append(0)

# Get the predictions
predictions = []
if '0' in result.measurements:
    predictions.append(result.measurements['0'][0])
else:
    predictions.append(0)

# Create the dates list
dates = pd.date_range(start=start_date, periods=30, freq='D')

# Create the predictions DataFrame
if len(predictions) < len(dates):
    dates = dates[:len(predictions)]
elif len(predictions) > len(dates):
    new_date = pd.date_range(start=dates[-1], periods=2, freq='D')[1]
    dates = dates.append(new_date)
df_pred = pd.DataFrame(predictions, index=dates, columns=['predictions'])

# Print the predictions DataFrame
print(df_pred)

# Create the circuit diagram
circuit_diagram = SVGCircuit(circuit)
print(circuit_diagram)


In [ ]:
import cirq
import pandas as pd
from datetime import datetime
from fredapi import Fred
from cirq.contrib.svg import SVGCircuit
import fredpy as fp



fp.api_key = fp.load_api_key('cucumber.txt')
fp = Fred(api_key=fp.api_key)

api_key = fp.api_key

# FRED series ID to retrieve data for
series_id = 'SP500'

def simulate_circuit(series_id, num_shots):
    # Download data from FRED
    fred = Fred(api_key=api_key)
    data = fred.get_series(series_id)
    dates = [d.strftime('%Y-%m-%d') for d in data.index]

    # Initialize circuit
    num_qubits = len(dates)
    circuit = cirq.Circuit()

    # Add Hadamard gates to create a uniform superposition
    for i in range(num_qubits):
        qubit = cirq.GridQubit(0, i)
        circuit.append(cirq.H(qubit))

    # Add phase flip gates based on the data
    for i, datum in enumerate(data):
        qubit = cirq.GridQubit(0, i)
        if datum > data.mean():
            circuit.append(cirq.Z(qubit))

    # Add measurement gates to the circuit
    for i in range(num_qubits):
        qubit = cirq.GridQubit(0, i)
        circuit.append(cirq.measure(qubit, key='m{}'.format(i)))

    # Simulate the circuit
    simulator = cirq.Simulator()
    result = simulator.run(circuit, repetitions=num_shots)

    # Calculate probabilities for each date
    probs = []
    for i in range(num_qubits):
        if 'm{}'.format(i) in result.histograms:
            probs.append(result.histogram(key='m')[i] / num_shots)
        else:
            probs.append(0)

    # Create Pandas DataFrame with results
    df_pred = pd.DataFrame(probs, index=dates, columns=['probabilities'])
    return df_pred
df = simulate_circuit('SP500', 1000)
print(df)

In [ ]:
import cirq
import pandas as pd
from datetime import datetime
from fredapi import Fred
from cirq.contrib.svg import SVGCircuit
import fredpy as fp



fp.api_key = fp.load_api_key('cucumber.txt')
fp = Fred(api_key=fp.api_key)

api_key = fp.api_key



def simulate_circuit(series_id, num_shots):
    # Download data from FRED
    fred = Fred(api_key=api_key)
    data = fred.get_series(series_id)
    dates = [d.strftime('%Y-%m-%d') for d in data.index]

    # Initialize circuit
    num_qubits = len(dates)
    circuit = cirq.Circuit()

    # Add Hadamard gates to create a uniform superposition
    for i in range(num_qubits):
        qubit = cirq.GridQubit(0, i)
        circuit.append(cirq.H(qubit))

    # Add phase flip gates based on the data
    for i, datum in enumerate(data):
        qubit = cirq.GridQubit(0, i)
        if datum > data.mean():
            circuit.append(cirq.Z(qubit))

    # Add measurement gates to the circuit
    for i in range(num_qubits):
        qubit = cirq.GridQubit(0, i)
        circuit.append(cirq.measure(qubit, key='m{}'.format(i)))

    # Simulate the circuit
    simulator = cirq.Simulator()
    result = simulator.run(circuit, repetitions=num_shots)

    # Calculate probabilities for each date
    probs = []
    for i in range(num_qubits):
        if 'm{}'.format(i) in result.measurements.keys():
            measurements = result.measurements['m{}'.format(i)]
            hist = measurements.histogram(key='result')
            counts = hist.to_dict()
            probs.append(counts[0] / num_shots)
        else:
            probs.append(0)
        
    # Create Pandas DataFrame with results
    df_pred = pd.DataFrame(probs, index=dates, columns=['probabilities'])
    return df_pred
df = simulate_circuit('SP500', 1000)
print(df)

In [ ]:
import cirq
import numpy as np
import pandas as pd
from datetime import datetime
import requests
import json
import matplotlib.pyplot as plt


def get_fred_data(series_id, api_key):
    """
    This function retrieves data for a given series ID from the FRED API and returns it as a Pandas DataFrame.
    """
    base_url = 'https://api.stlouisfed.org/fred/series/observations?series_id={}&api_key={}'.format(series_id, api_key)
    response = requests.get(base_url)
    data = json.loads(response.text)['observations']
    dates = []
    values = []
    for observation in data:
        dates.append(datetime.strptime(observation['date'], '%Y-%m-%d').date())
        values.append(float(observation['value']))
    df = pd.DataFrame({'value': values}, index=dates)
    return df


def get_predictions(qubit, length):
    """
    This function generates a Cirq circuit to predict a binary time series based on a given input qubit, and returns
    the predictions as a list of percentages.
    """
    circuit = cirq.Circuit(
        cirq.H(qubit),
        cirq.measure(qubit, key='0')
    )
    predictions = []
    for i in range(length):
        simulator = cirq.Simulator()
        result = simulator.run(circuit, repetitions=100)
        count = result.histogram(key='0')
        predictions.append(count[1] / sum(count))
    return predictions


def simulate_and_predict(df, qubit):
    """
    This function generates a series of binary predictions for a given input DataFrame and qubit.
    """
    predictions = []
    for i in range(len(df)):
        if i == 0:
            # Initialize with a uniform superposition
            circuit = cirq.Circuit(cirq.X(qubit), cirq.H(qubit))
        else:
            # Condition on the previous value
            if df.iloc[i-1]['value'] == 1:
                circuit = cirq.Circuit(cirq.X(qubit), cirq.H(qubit))
            else:
                circuit = cirq.Circuit(cirq.H(qubit))
        result = cirq.Simulator().run(circuit, repetitions=100)
        count = result.histogram(key='0')
        prediction = count[1] / sum(count)
        predictions.append(prediction)
    return predictions


def analyze_results(results, qubits):
    """
    This function analyzes the results of a set of circuit simulations and returns a histogram of the number of times each
    qubit was measured in the |1> state.
    """
    histograms = []
    for i, qubit in enumerate(qubits):
        m_results = []
        for result in results:
            counts = result.measurements[str(qubit)]
            m_result = np.count_nonzero(counts)
            m_results.append(m_result)
        histograms.append(np.histogram(m_results, bins=range(len(results) + 1))[0])
    return histograms


def plot_predictions(df, predictions):
    """
    This function plots the predictions along with the original time series data.
    """
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.fill_between(df.index, 0, 1, where=df['value'] == 1, color='b', alpha=0.1, interpolate=True)
    ax.fill_between(df.index, 0, 1, where=df['value'] == 0, color='r', alpha=0.1, interpolate=True)
    ax.plot(df.index, predictions, label='predictions')
    ax.legend(loc='best')
    plt.show()


def main():
    # Define the FRED series ID and number of qubits
    series_id = "GDPC1"
    num_qubits = 8

    # Convert FRED data to qubits
    circuit = fred_data_to_qubits(series_id, num_qubits)

    # Simulate the circuit
    num_simulations = 1000
    percentages = simulate_circuit(circuit, num_simulations)

    # Create a Pandas DataFrame with the simulation results
    start_date = datetime.now() - timedelta(days=num_simulations)
    dates = pd.date_range(start_date, periods=num_simulations, freq="D")
    df_pred = pd.DataFrame(percentages, index=dates, columns=["Percentage"])

    # Plot the results
    fig, ax = plt.subplots()
    ax.plot(df_pred.index, df_pred["Percentage"])
    ax.fill_between(df_pred.index, 0, df_pred["Percentage"], alpha=0.1)
    ax.set_xlabel("Date")
    ax.set_ylabel("Percentage")
    ax.set_title("Prediction of FRED Data")
    plt.show()

if __name__ == "__main__":
    main()


In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np

def create_model(input_shape):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(16, input_shape=input_shape, activation='relu'))
    model.add(tf.keras.layers.Dense(8, activation='relu'))
    model.add(tf.keras.layers.Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model

def predict_time_series(data, future_dates):
    
    # Split data into training and testing sets
    train_size = int(len(data) * 0.8)
    train_data = data[:train_size]
    test_data = data[train_size:]

    # Normalize the data
    train_mean = train_data.mean()
    train_std = train_data.std()
    train_data = (train_data - train_mean) / train_std
    test_data = (test_data - train_mean) / train_std

    # Create input and output sequences for the model
    def create_sequences(data, window_size):
        xs = []
        ys = []
        for i in range(window_size, len(data)):
            xs.append(data[i-window_size:i])
            ys.append(data[i])
        return tf.convert_to_tensor(xs), tf.convert_to_tensor(ys)

    window_size = 10
    train_xs, train_ys = create_sequences(train_data, window_size)
    test_xs, test_ys = create_sequences(test_data, window_size)

    # Train the model
    model = create_model((window_size,))
    history = model.fit(train_xs, train_ys, epochs=50, verbose=0)

    # Make predictions
    future_data = data[-window_size:]
    future_data = (future_data - train_mean) / train_std
    for date in future_dates:
        prediction = model.predict(tf.convert_to_tensor(future_data.reshape((1, window_size))))
        data = data.append(pd.Series(prediction[0]), ignore_index=True)
        future_data = data[-window_size:]
        future_data = (future_data - train_mean) / train_std

    return data

data = pd.read_csv('input.csv')

future_dates = ['2023-09-09', '2023-12-10', '2024-04-11']

predicted_data = predict_time_series(data, future_dates)
print(predicted_data.tail(len(future_dates)))


In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


def create_model(input_shape):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(16, input_shape=input_shape, activation='relu'))
    model.add(tf.keras.layers.Dense(8, activation='relu'))
    model.add(tf.keras.layers.Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model

def create_sequences(data, window_size):
    X = []
    y = []
    for i in range(len(data)-window_size):
        X.append(data[i:i+window_size])
        y.append(data[i+window_size])
    return np.array(X), np.array(y)

def predict_time_series(data, future_dates):
    # Load data
    data = pd.read_csv('input.csv')
    
    # Split data into training and testing sets
    train_size = int(len(data) * 0.8)
    train_data = data[:train_size]
    test_data = data[train_size:]

    # Normalize the data
    train_mean = train_data.mean()
    train_std = train_data.std()
    train_data = (train_data - train_mean) / train_std
    test_data = (test_data - train_mean) / train_std

    # Create input and output sequences for the model
    window_size = 10
    train_xs, train_ys = create_sequences(train_data, window_size)
    test_xs, test_ys = create_sequences(test_data, window_size)

    # Train the model
    model = create_model((window_size,))
    history = model.fit(train_xs, train_ys, epochs=50, verbose=0)

    # Make predictions
    future_data = data[-window_size:]
    future_data = (future_data - train_mean) / train_std
    for date in future_dates:
        prediction = model.predict(tf.convert_to_tensor(future_data.reshape((1, window_size))))
        data = data.append(pd.Series(prediction[0]), ignore_index=True)
        future_data = data[-window_size:]
        future_data = (future_data - train_mean) / train_std

    return data

data = pd.read_csv('input.csv')

future_dates = ['2023-09-09', '2023-12-10', '2024-04-11']

predicted_data = predict_time_series(data, future_dates)
print(predicted_data.tail(len(future_dates)))

def plot_results(data, future_dates, predicted_data):
    # Create a figure
    fig = plt.figure(figsize=(12, 8))
    ax = fig.add_subplot(1, 1, 1)

    # Plot the actual data
    ax.plot(data.index, data.values, label='Actual')

    # Plot the projected data
    future_index = pd.date_range(start=data.index[-1], periods=len(future_dates)+1, freq='M')[1:]
    ax.plot(future_index, predicted_data[-len(future_dates):].values, label='Projected')

    # Set the title and axis labels
    ax.set_title('Projected Data')
    ax.set_xlabel('Date')
    ax.set_ylabel('Value')

    # Set the x-axis ticks to be every 3 months
    ax.xaxis.set_major_locator(plt.MaxNLocator(6))

    # Add a legend
    ax.legend()

    # Show the plot
    plt.show()

plot_results(data, future_dates, predicted_data)


In [ ]:
import cirq
import pandas as pd
from datetime import datetime
from fredapi import Fred
from cirq.contrib.svg import SVGCircuit
import fredpy as fp



fp.api_key = fp.load_api_key('cucumber.txt')
fp = Fred(api_key=fp.api_key)

api_key = fp.api_key




def simulate_circuit(series_id, num_shots):
    # Download data from FRED
    fred = Fred(api_key=api_key)
    data = fred.get_series(series_id)
    dates = [d.strftime('%Y-%m-%d') for d in data.index]

    # Initialize circuit
    num_qubits = len(dates)
    circuit = cirq.Circuit()

    # Add Hadamard gates to create a uniform superposition
    for i in range(num_qubits):
        qubit = cirq.GridQubit(0, i)
        circuit.append(cirq.H(qubit))

    # Add phase flip gates based on the data
    for i, datum in enumerate(data):
        qubit = cirq.GridQubit(0, i)
        if datum > data.mean():
            circuit.append(cirq.Z(qubit))

    # Add measurement gates to the circuit
    for i in range(num_qubits):
        qubit = cirq.GridQubit(0, i)
        circuit.append(cirq.measure(qubit, key='m{}'.format(i)))

    # Simulate the circuit
    simulator = cirq.Simulator()
    result = simulator.run(circuit, repetitions=num_shots)

    # Calculate probabilities for each date
    probs = []
    for i in range(num_qubits):
        if 'm{}'.format(i) in result.measurements.keys():
            measurements = result.measurements['m{}'.format(i)]
            hist = measurements.histogram(key='result')
            counts = hist.to_dict()
            probs.append(counts[0] / num_shots)
        else:
            probs.append(0)

    # Create Pandas DataFrame with results
    df_pred = pd.DataFrame(probs, index=dates, columns=['probabilities'])

    return df_pred
df = simulate_circuit('SP500', 1000)
print(df)

In [ ]:
#Simulate a quantum projection for the federal funds rate
import pandas as pd
import numpy as np
import cirq
import matplotlib.pyplot as plt
import requests
import json
import fredpy as fp

fp.api_key = fp.load_api_key('cucumber.txt')

api_key = fp.api_key
series_id = 'SP500'
num_steps = 500

# Request data from FRED API
url = f'https://api.stlouisfed.org/fred/series/observations?series_id={series_id}&api_key={api_key}&file_type=json'
response = requests.get(url)
data = json.loads(response.text)['observations']


# Clean data and prepare for prediction
df = pd.DataFrame(data)
df['date'] = pd.to_datetime(df['date'])
df['value'] = pd.to_numeric(df['value'], errors='coerce')
df.dropna(inplace=True)
df.set_index('date', inplace=True)
start_date = df.index[-1]
end_date = start_date + pd.DateOffset(days=num_steps-1)  # Subtract 1 from num_steps
dates = pd.date_range(start_date, end_date)

#Define Qubits
num_qubits = len(dates)
qubit = cirq.GridQubit(0, 0)

# Add measurement gate to circuit
circuit = cirq.Circuit()
circuit.append(cirq.measure(qubit))
simulator = cirq.Simulator()
# Run simulation
result = simulator.run(circuit, repetitions=100)

# Define quantum circuit and simulator

circuit = cirq.Circuit(cirq.X(qubit), cirq.measure(qubit)) # Add measurement


# Define function to generate predictions
def generate_predictions(steps):
    predictions = []
    for i in range(steps):
        simulator = cirq.Simulator()
        result = simulator.run(circuit, repetitions=100)
        count = result.histogram(key=str(q))
        predictions.append(count[1] / sum(count))
    return predictions


# Generate predictions
predictions = generate_predictions(num_steps)

# Calculate probability ranges
df_pred = pd.DataFrame(predictions, index=dates, columns=['predictions'])
df_prob = df_pred.groupby(pd.Grouper(freq='W')).agg(['mean', 'std'])
df_prob['ci_lower'] = df_prob[('predictions', 'mean')] - df_prob[('predictions', 'std')]
df_prob['ci_upper'] = df_prob[('predictions', 'mean')] + df_prob[('predictions', 'std')]

# Plot predictions with fill_between style
plt.fill_between(df_prob.index, df_prob['ci_lower'], df_prob['ci_upper'], alpha=0.2)
plt.plot(df.index, df['value'], label=series_id)
plt.legend()
# Add axis labels and title
plt.xlabel('Dates of Observation')
plt.ylabel(series_id)
plt.title(series_id)
plt.show()

# Print probability ranges as Pandas data frame
print(df_prob)


In [ ]:
#Function to generate predictions. Ideally this would be fitten to the historical data points. 
#Log linear power laws just smooth out the series, we want functions with more predictive power

def generate_predictions(steps):
    predictions = []
    for i in range(length):
        simulator = cirq.Simulator()
        result = simulator.run(circuit, repetitions=100)
        counts = result.histogram(key='0')[0]
        predictions.append(counts[1] / sum(counts))
    return predictions
